# FINANCIAL FRAUD DETECTION USING AI , ML

In [2]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier

In [3]:
tr_df = pd.read_csv("./transactions.csv")

In [4]:
tr_df.columns

Index(['Unnamed: 0', 'accountNumber', 'customerId', 'creditLimit',
       'availableMoney', 'transactionDateTime', 'transactionAmount',
       'merchantName', 'acqCountry', 'merchantCountryCode', 'posEntryMode',
       'posConditionCode', 'merchantCategoryCode', 'currentExpDate',
       'accountOpenDate', 'dateOfLastAddressChange', 'cardCVV', 'enteredCVV',
       'cardLast4Digits', 'transactionType', 'echoBuffer', 'currentBalance',
       'merchantCity', 'merchantState', 'merchantZip', 'cardPresent',
       'posOnPremises', 'recurringAuthInd', 'expirationDateKeyInMatch',
       'isFraud'],
      dtype='object')

In [5]:
# tr_df.shape
tr_df.describe()

,Unnamed: 0,accountNumber,customerId,creditLimit,availableMoney,transactionAmount,posEntryMode,posConditionCode,cardCVV,enteredCVV,cardLast4Digits,echoBuffer,currentBalance,merchantCity,merchantState,merchantZip,posOnPremises,recurringAuthInd
count,786363.000000,7.863630e+05,7.863630e+05,786363.000000,786363.000000,786363.000000,782309.000000,785954.000000,786363.000000,786363.000000,786363.000000,0.0,786363.000000,0.0,0.0,0.0,0.0,0.0
mean,393181.000000,5.372326e+08,5.372326e+08,10759.464459,6250.725369,136.985791,9.049943,3.271980,544.467338,544.183857,4757.417799,NaN,4508.739089,NaN,NaN,NaN,NaN,NaN
std,227003.589203,2.554211e+08,2.554211e+08,11636.174890,8880.783989,147.725569,16.774130,9.809023,261.524220,261.551254,2996.583810,NaN,6457.442068,NaN,NaN,NaN,NaN,NaN
min,0.000000,1.000881e+08,1.000881e+08,250.000000,-1005.630000,0.000000,2.000000,1.000000,100.000000,0.000000,0.000000,NaN,0.000000,NaN,NaN,NaN,NaN,NaN
25%,196590.500000,3.301333e+08,3.301333e+08,5000.000000,1077.420000,33.650000,2.000000,1.000000,310.000000,310.000000,2178.000000,NaN,689.910000,NaN,NaN,NaN,NaN,NaN
50%,393181.000000,5.074561e+08,5.074561e+08,7500.000000,3184.860000,87.900000,5.000000,1.000000,535.000000,535.000000,4733.000000,NaN,2451.760000,NaN,NaN,NaN,NaN,NaN
75%,589771.500000,7.676200e+08,7.676200e+08,15000.000000,7500.000000,191.480000,9.000000,1.000000,785.000000,785.000000,7338.000000,NaN,5291.095000,NaN,NaN,NaN,NaN,NaN
max,786362.000000,9.993896e+08,9.993896e+08,50000.000000,50000.000000,2011.540000,90.000000,99.000000,998.000000,998.000000,9998.000000,NaN,47498.810000,NaN,NaN,NaN,NaN,NaN


In [6]:
tr_df.head(5)

,Unnamed: 0,accountNumber,customerId,creditLimit,availableMoney,transactionDateTime,transactionAmount,merchantName,acqCountry,merchantCountryCode,...,echoBuffer,currentBalance,merchantCity,merchantState,merchantZip,cardPresent,posOnPremises,recurringAuthInd,expirationDateKeyInMatch,isFraud
0,0,737265056,737265056,5000,5000.0,2016-08-13T14:27:32,98.55,Uber,US,US,...,NaN,0.0,NaN,NaN,NaN,False,NaN,NaN,False,False
1,1,737265056,737265056,5000,5000.0,2016-10-11T05:05:54,74.51,AMC #191138,US,US,...,NaN,0.0,NaN,NaN,NaN,True,NaN,NaN,False,False
2,2,737265056,737265056,5000,5000.0,2016-11-08T09:18:39,7.47,Play Store,US,US,...,NaN,0.0,NaN,NaN,NaN,False,NaN,NaN,False,False
3,3,737265056,737265056,5000,5000.0,2016-12-10T02:14:50,7.47,Play Store,US,US,...,NaN,0.0,NaN,NaN,NaN,False,NaN,NaN,False,False
4,4,830329091,830329091,5000,5000.0,2016-03-24T21:04:46,71.18,Tim Hortons #947751,US,US,...,NaN,0.0,NaN,NaN,NaN,True,NaN,NaN,False,False


In [7]:
tr_df = tr_df.drop(['recurringAuthInd','posOnPremises','merchantZip','merchantState','merchantCity','echoBuffer'], axis=1)

In [8]:
tr_df.isnull().sum()

Unnamed: 0                     0
accountNumber                  0
customerId                     0
creditLimit                    0
availableMoney                 0
transactionDateTime            0
transactionAmount              0
merchantName                   0
acqCountry                  4562
merchantCountryCode          724
posEntryMode                4054
posConditionCode             409
merchantCategoryCode           0
currentExpDate                 0
accountOpenDate                0
dateOfLastAddressChange        0
cardCVV                        0
enteredCVV                     0
cardLast4Digits                0
transactionType              698
currentBalance                 0
cardPresent                    0
expirationDateKeyInMatch       0
isFraud                        0
dtype: int64

In [9]:
tr_df.fillna(method="ffill", inplace=True)
tr_df.isnull().sum()

C:\Users\DeLL\AppData\Local\Temp\ipykernel_2000\816393495.py:1: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  tr_df.fillna(method="ffill", inplace=True)


Unnamed: 0                  0
accountNumber               0
customerId                  0
creditLimit                 0
availableMoney              0
transactionDateTime         0
transactionAmount           0
merchantName                0
acqCountry                  0
merchantCountryCode         0
posEntryMode                0
posConditionCode            0
merchantCategoryCode        0
currentExpDate              0
accountOpenDate             0
dateOfLastAddressChange     0
cardCVV                     0
enteredCVV                  0
cardLast4Digits             0
transactionType             0
currentBalance              0
cardPresent                 0
expirationDateKeyInMatch    0
isFraud                     0
dtype: int64

# abstract useful data in datafram another no need fraud detection

In [32]:
df = tr_df[[
    'accountNumber',
    'customerId',
    'creditLimit',
    'availableMoney',
    'transactionDateTime',
    'transactionAmount',
    'merchantName',
    'acqCountry',
    'posEntryMode',
    'posConditionCode',
    'merchantCategoryCode',
    'transactionType',
    'currentBalance',
    'cardPresent',
    'isFraud'
]]

In [33]:
df.head(5)

,accountNumber,customerId,creditLimit,availableMoney,transactionDateTime,transactionAmount,merchantName,acqCountry,posEntryMode,posConditionCode,merchantCategoryCode,transactionType,currentBalance,cardPresent,isFraud
0,737265056,737265056,5000,5000.0,2016-08-13T14:27:32,98.55,Uber,US,2.0,1.0,rideshare,PURCHASE,0.0,False,False
1,737265056,737265056,5000,5000.0,2016-10-11T05:05:54,74.51,AMC #191138,US,9.0,1.0,entertainment,PURCHASE,0.0,True,False
2,737265056,737265056,5000,5000.0,2016-11-08T09:18:39,7.47,Play Store,US,9.0,1.0,mobileapps,PURCHASE,0.0,False,False
3,737265056,737265056,5000,5000.0,2016-12-10T02:14:50,7.47,Play Store,US,9.0,1.0,mobileapps,PURCHASE,0.0,False,False
4,830329091,830329091,5000,5000.0,2016-03-24T21:04:46,71.18,Tim Hortons #947751,US,2.0,1.0,fastfood,PURCHASE,0.0,True,False


In [34]:
df['transactionDateTime'] = pd.to_datetime(df['transactionDateTime'])
df['year'] = df['transactionDateTime'].dt.year
df['month'] = df['transactionDateTime'].dt.month
df['day'] = df['transactionDateTime'].dt.day
df['hour'] = df['transactionDateTime'].dt.hour
df['day_of_week'] = df['transactionDateTime'].dt.dayofweek


C:\Users\DeLL\AppData\Local\Temp\ipykernel_2000\3906669979.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['transactionDateTime'] = pd.to_datetime(df['transactionDateTime'])
C:\Users\DeLL\AppData\Local\Temp\ipykernel_2000\3906669979.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['year'] = df['transactionDateTime'].dt.year
C:\Users\DeLL\AppData\Local\Temp\ipykernel_2000\3906669979.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[r

In [35]:
transaction_counts = df.groupby('customerId')['transactionAmount'].count().reset_index()
transaction_counts.columns = ['customerId', 'transaction_count']
df = df.merge(transaction_counts, on='customerId', how='left')

In [36]:
# Example: Average transaction amount per user
average_transaction_amount = df.groupby('customerId')['transactionAmount'].mean().reset_index()
average_transaction_amount.columns = ['customerId', 'avg_transaction_amount']
df = df.merge(average_transaction_amount, on='customerId', how='left')

In [37]:
df.head(5)

,accountNumber,customerId,creditLimit,availableMoney,transactionDateTime,transactionAmount,merchantName,acqCountry,posEntryMode,posConditionCode,...,currentBalance,cardPresent,isFraud,year,month,day,hour,day_of_week,transaction_count,avg_transaction_amount
0,737265056,737265056,5000,5000.0,2016-08-13 14:27:32,98.55,Uber,US,2.0,1.0,...,0.0,False,False,2016,8,13,14,5,4,47.000000
1,737265056,737265056,5000,5000.0,2016-10-11 05:05:54,74.51,AMC #191138,US,9.0,1.0,...,0.0,True,False,2016,10,11,5,1,4,47.000000
2,737265056,737265056,5000,5000.0,2016-11-08 09:18:39,7.47,Play Store,US,9.0,1.0,...,0.0,False,False,2016,11,8,9,1,4,47.000000
3,737265056,737265056,5000,5000.0,2016-12-10 02:14:50,7.47,Play Store,US,9.0,1.0,...,0.0,False,False,2016,12,10,2,5,4,47.000000
4,830329091,830329091,5000,5000.0,2016-03-24 21:04:46,71.18,Tim Hortons #947751,US,2.0,1.0,...,0.0,True,False,2016,3,24,21,3,11,107.255455


In [38]:
# Get all string (object) columns
string_columns = df.select_dtypes(include=['object']).columns.tolist()
print("String columns:", string_columns)


String columns: ['merchantName', 'acqCountry', 'merchantCategoryCode', 'transactionType']


In [39]:
df[['merchantName', 'acqCountry', 'merchantCategoryCode', 'transactionType']].head(5)

,merchantName,acqCountry,merchantCategoryCode,transactionType
0,Uber,US,rideshare,PURCHASE
1,AMC #191138,US,entertainment,PURCHASE
2,Play Store,US,mobileapps,PURCHASE
3,Play Store,US,mobileapps,PURCHASE
4,Tim Hortons #947751,US,fastfood,PURCHASE


In [40]:
df['acqCountry'].value_counts()

acqCountry
US     779227
MEX      3149
CAN      2441
PR       1546
Name: count, dtype: int64

In [41]:
from sklearn.preprocessing import LabelEncoder
# Initialize the label encoder
le = LabelEncoder()

# Loop through each column and apply label encoding
for col in string_columns:
    df[col] = le.fit_transform(df[col])

In [42]:
df['isFraud'] = df['isFraud'].astype(int)
df['cardPresent'] = df['cardPresent'].astype(int)

In [43]:
df.head(5)

,accountNumber,customerId,creditLimit,availableMoney,transactionDateTime,transactionAmount,merchantName,acqCountry,posEntryMode,posConditionCode,...,currentBalance,cardPresent,isFraud,year,month,day,hour,day_of_week,transaction_count,avg_transaction_amount
0,737265056,737265056,5000,5000.0,2016-08-13 14:27:32,98.55,2086,3,2.0,1.0,...,0.0,0,0,2016,8,13,14,5,4,47.000000
1,737265056,737265056,5000,5000.0,2016-10-11 05:05:54,74.51,27,3,9.0,1.0,...,0.0,1,0,2016,10,11,5,1,4,47.000000
2,737265056,737265056,5000,5000.0,2016-11-08 09:18:39,7.47,1305,3,9.0,1.0,...,0.0,0,0,2016,11,8,9,1,4,47.000000
3,737265056,737265056,5000,5000.0,2016-12-10 02:14:50,7.47,1305,3,9.0,1.0,...,0.0,0,0,2016,12,10,2,5,4,47.000000
4,830329091,830329091,5000,5000.0,2016-03-24 21:04:46,71.18,2084,3,2.0,1.0,...,0.0,1,0,2016,3,24,21,3,11,107.255455


In [44]:
normal_data = df[df['isFraud']==0]
fraud_data = df[df['isFraud']==1]

In [45]:
normal_data.shape,fraud_data.shape

((773946, 22), (12417, 22))

In [46]:
fraud_row = fraud_data.shape[0]

In [47]:
sample_data = normal_data.sample(fraud_row)
df = pd.concat([sample_data,fraud_data])

In [48]:
df['isFraud'].value_counts()

isFraud
0    12417
1    12417
Name: count, dtype: int64

In [49]:
x = df.drop(['isFraud','transactionDateTime','merchantName'],axis=1)
y = df[['isFraud']]

In [52]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.8, random_state=42)
y_train.value_counts()

isFraud
1          2518
0          2448
Name: count, dtype: int64

In [53]:
scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(x_train)
x_test_scaled = scaler.transform(x_test)  # Scale your test set as well

In [54]:
dt_classifier = DecisionTreeClassifier(criterion='gini', random_state=42, class_weight='balanced')
dt_classifier.fit(x_train_scaled, y_train)

DecisionTreeClassifier(class_weight='balanced', random_state=42)

In [55]:
y_pred = dt_classifier.predict(x_test_scaled)

In [56]:
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy * 100:.2f}%")

Accuracy: 61.50%


    Accuracy: 95.05%    

In [57]:
conf_matrix = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:")
print(conf_matrix)

Confusion Matrix:
[[6144 3825]
 [3824 6075]]


# Confusion Matrix:
    [[372893  14172]
    [  5297    820]]

In [58]:
class_report = classification_report(y_test, y_pred)
print("Classification Report:")
print(class_report)

Classification Report:
              precision    recall  f1-score   support

           0       0.62      0.62      0.62      9969
           1       0.61      0.61      0.61      9899

    accuracy                           0.62     19868
   macro avg       0.62      0.62      0.62     19868
weighted avg       0.62      0.62      0.62     19868



Classification Report:
              precision    recall  f1-score   support

           0       0.99      0.96      0.97    387065
           1       0.05      0.13      0.08      6117

      accuracy                           0.95    393182
      macro avg       0.52      0.55      0.53    393182
      weighted avg       0.97      0.95      0.96    393182

In [59]:
knn = KNeighborsClassifier(n_neighbors=5)

# Fit the model on the training data
knn.fit(x_train_scaled, y_train)

# Make predictions on the test data
y_pred = knn.predict(x_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy * 100:.2f}%")

# Confusion Matrix
conf_matrix = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:")
print(conf_matrix)

# Classification Report (Precision, Recall, F1-Score)
class_report = classification_report(y_test, y_pred)
print("Classification Report:")
print(class_report)


c:\Users\DeLL\python\Lib\site-packages\sklearn\neighbors\_classification.py:238: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
c:\Users\DeLL\python\Lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but KNeighborsClassifier was fitted without feature names
  warnings.warn(


Accuracy: 55.00%
Confusion Matrix:
[[6186 3783]
 [5157 4742]]
Classification Report:
              precision    recall  f1-score   support

           0       0.55      0.62      0.58      9969
           1       0.56      0.48      0.51      9899

    accuracy                           0.55     19868
   macro avg       0.55      0.55      0.55     19868
weighted avg       0.55      0.55      0.55     19868



In [60]:


# Initialize the model
model = Sequential()

# Input layer and first hidden layer
model.add(Dense(units=64, activation='relu', input_shape=(x_train_scaled.shape[1],)))

# Second hidden layer
model.add(Dense(units=32, activation='relu'))

# Optional: Adding dropout to prevent overfitting
model.add(Dropout(0.5))

# Output layer (binary classification)
model.add(Dense(units=1, activation='sigmoid'))

# Compile the model
model.compile(optimizer ='adam', loss='binary_crossentropy', metrics=['accuracy'])


c:\Users\DeLL\python\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [61]:
# Early stopping to prevent overfitting
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

# Train the model
history = model.fit(
    x_train_scaled, y_train, 
    validation_split=0.2, 
    epochs=4, 
    batch_size=64, 
    callbacks=[early_stopping]
)


Epoch 1/4
63/63 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - accuracy: 0.4901 - loss: 0.7348 - val_accuracy: 0.6127 - val_loss: 0.6658
Epoch 2/4
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5964 - loss: 0.6609 - val_accuracy: 0.6449 - val_loss: 0.6523
Epoch 3/4
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6090 - loss: 0.6523 - val_accuracy: 0.6499 - val_loss: 0.6487
Epoch 4/4
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6088 - loss: 0.6514 - val_accuracy: 0.6590 - val_loss: 0.6449


In [62]:
# Evaluate on test data
loss, accuracy = model.evaluate(x_test_scaled, y_test)
print(f'Test Accuracy: {accuracy:.4f}')

# Predictions on test data
y_pred = (model.predict(x_test_scaled) > 0.5).astype("int32")


621/621 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.6348 - loss: 0.6435
Test Accuracy: 0.6367
621/621 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step


In [63]:
from sklearn.metrics import classification_report

print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

           0       0.63      0.65      0.64      9969
           1       0.64      0.62      0.63      9899

    accuracy                           0.64     19868
   macro avg       0.64      0.64      0.64     19868
weighted avg       0.64      0.64      0.64     19868

